In [1]:
import pandas as pd
from docx.api import Document
import pandas as pd
import re
import os
import numpy as np
import tabula

In [2]:
def extract_mapping(file_name):
    graduate_list = tabula.read_pdf(f"pdfs/{file_name}.pdf", pages=[1,2,3,4,5,6], multiple_tables=True, relative_area=True)

    exam_number_mapping = {}
    for table in graduate_list:
        if len(table.columns) == 3:  # in case of viable mapping table
            if list(table[1][table[1].str.contains('B. Sc.|Bachelor') == True]):
                extract_df = table.dropna()[[0,1]]
                exam_number_mapping['Bachelor'] = {row[0].replace(' ',''): row[1]  for row in extract_df.to_dict('records')}
            elif list(table[1][table[1].str.contains('M. Sc.|Master') == True]):
                extract_df = table.dropna()[[0,1]]
                exam_number_mapping['Master'] = {row[0].replace(' ',''): row[1]  for row in extract_df.to_dict('records')}

        elif len(table.columns) == 4:
            if list(table[1][table[1].str.contains('B. Sc.|Bachelor') == True]):
                extract_df = table[[0,1]].dropna()
                exam_number_mapping['Bachelor'] = {row[0].replace(' ',''): row[1]  for row in extract_df.to_dict('records')}
            elif list(table[1][table[1].str.contains('M. Sc.|Master') == True]):
                extract_df = table[[0,1]].dropna()
                exam_number_mapping['Master'] = {row[0].replace(' ',''): row[1]  for row in extract_df.to_dict('records')}                
                
        elif len(table.columns) == 5:
            if list(table[2][table[2].str.contains('B. Sc.|Bachelor') == True]):
                extract_df = table[[0,2]].dropna()
                exam_number_mapping['Bachelor'] = {row[0].replace(' ',''): row[2]  for row in extract_df.to_dict('records')}
            elif list(table[2][table[2].str.contains('M. Sc.|Master') == True]):
                extract_df = table[[0,2]].dropna()
                exam_number_mapping['Master'] = {row[0].replace(' ',''): row[2]  for row in extract_df.to_dict('records')}
                       
    return exam_number_mapping

In [3]:
path = 'word_docs'
total_mapping = {}
file_names = os.listdir(path)
total_graduate_data = pd.DataFrame()
na_counter = 0
for file_name in reversed(file_names):
    print(file_name)
    document = Document(f'word_docs/{file_name}')
    exam_year = re.search(r'(\d+)', file_name).group(1)
    exam_semester = re.search(r'^(.+)_\d+\.docx', file_name).group(1)
    
    word_table = document.tables
    mapping = extract_mapping(file_name=re.search(r'(.+)\.docx', file_name).group(1))
    total_mapping.update(mapping)  # add new Prüfungsnummer
    table_count = 0
    for table in word_table:
        # convert to pandas
        table_list = [[cell.text for cell in row.cells] for row in table.rows]
        table_df = pd.DataFrame(table_list)
        if len(table_df.columns) == 11:  # in case of viable grade-table
            # extract exam number from table
            try:
                exam_number = re.search(r'(\d+)', table_df[0][0]).group(1)
            except:
                exam_number = np.nan
                na_counter += 1
            # restrict to only those rows that contain grades
            table_df = table_df[table_df[1].str.contains(r'\d,\d')][[1, 2, 5, 8]]
            table_df.columns = ['Grade', 'first_exam', 'second_exam', 'total']  # rename

            table_df['exam_year'] = exam_year
            table_df['exam_semester'] = exam_semester
            table_df['exam_number'] = exam_number
            # map course onto exam_number
            try:
                table_df['course'] = total_mapping['Bachelor'][exam_number]
                table_df['degree'] = 'Bachelor'
            except:
                try:
                    table_df['course'] = total_mapping['Master'][exam_number]
                    table_df['degree'] = 'Master'
                except:
                    table_df['course'] = np.nan
                    table_df['degree'] = np.nan
            table_count += 1
            total_graduate_data = pd.concat([total_graduate_data, table_df])
    print(f'Number of tables: {table_count}')

print(f'Non-available Prüfungsnummer: {na_counter}')

Wintersemester_2017.docx
Number of tables: 44
Wintersemester_2016.docx
Number of tables: 46
Wintersemester_2015.docx
Number of tables: 40
Wintersemester_2014.docx
Number of tables: 41
Wintersemester_2013.docx
Number of tables: 40
Wintersemester_2012.docx
Number of tables: 39
Sommersemester_2018.docx
Number of tables: 43
Sommersemester_2017.docx
Number of tables: 40
Sommersemester_2016.docx
Number of tables: 40
Sommersemester_2015.docx
Number of tables: 40
Sommersemester_2014.docx
Number of tables: 40
Sommersemester_2013.docx
Number of tables: 39
Non-available Prüfungsnummer: 1


In [4]:
total_graduate_data[total_graduate_data['first_exam'].str.contains('^\n', regex=True)==True]

,Grade,first_exam,second_exam,total,exam_year,exam_semester,exam_number,course,degree
3,"1,0",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,3,2017,Wintersemester,332224027,Corporate Finance,Master
4,"1,3",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,4,2017,Wintersemester,332224027,Corporate Finance,Master
5,"1,7",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,2,2017,Wintersemester,332224027,Corporate Finance,Master
6,"2,0",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,1,2017,Wintersemester,332224027,Corporate Finance,Master
7,"2,3",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,1,2017,Wintersemester,332224027,Corporate Finance,Master
8,"2,7",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,2,2017,Wintersemester,332224027,Corporate Finance,Master
9,"3,0",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,0,2017,Wintersemester,332224027,Corporate Finance,Master
10,"3,3",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,0,2017,Wintersemester,332224027,Corporate Finance,Master
11,"3,7",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,0,2017,Wintersemester,332224027,Corporate Finance,Master
12,"4,0",\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,\n\n\n\n\n\n\n\nAus datenschutzrechtlichen Grü...,0,2017,Wintersemester,332224027,Corporate Finance,Master


In [5]:
# clean data
total_graduate_data['first_exam'][total_graduate_data['first_exam'].str.contains('^\n', regex=True)==True] = np.nan
total_graduate_data['second_exam'][total_graduate_data['second_exam'].str.contains('^\n', regex=True)==True] = np.nan
total_graduate_data['total'][total_graduate_data['total'].str.contains('^\n', regex=True)==True] = np.nan
# convert to numeric
total_graduate_data[["first_exam", "second_exam", "total"]] = total_graduate_data[["first_exam", "second_exam", "total"]].apply(pd.to_numeric)

In [6]:
total_graduate_data[total_graduate_data['total'].isnull()]

,Grade,first_exam,second_exam,total,exam_year,exam_semester,exam_number,course,degree
3,"1,0",18.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
4,"1,3",15.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
5,"1,7",21.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
6,"2,0",9.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
7,"2,3",4.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
8,"2,7",3.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
9,"3,0",0.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
10,"3,3",0.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
11,"3,7",0.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor
12,"4,0",0.0,NaN,NaN,2017,Wintersemester,333210050,Wissenschaftliches Arbeiten,Bachelor


In [7]:
# make column 'total' s.t. it is always sum of 'first_exam' and 'second_exam'
total_graduate_data['total'] = np.select(
                                            [
                                                total_graduate_data['total'].isnull(), 
                                                ~total_graduate_data['total'].isnull()
                                            ], 
                                            [
                                                total_graduate_data['first_exam'].add(total_graduate_data['second_exam'], fill_value=0), 
                                                total_graduate_data['total']
                                            ], 
                                                total_graduate_data['total']
                                        )

In [8]:
# no data was available for these courses
total_graduate_data[total_graduate_data['total'].isnull()]

,Grade,first_exam,second_exam,total,exam_year,exam_semester,exam_number,course,degree
3,"1,0",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
4,"1,3",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
5,"1,7",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
6,"2,0",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
7,"2,3",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
8,"2,7",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
9,"3,0",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
10,"3,3",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
11,"3,7",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor
12,"4,0",NaN,NaN,NaN,2018,Sommersemester,333210002,Grundzüge der VWL: Einführung in die Makroökon...,Bachelor


In [9]:
total_graduate_data.to_excel('grade_stats.xlsx', index=False)